In [2]:
from pyspark.sql import functions as F

yellow_path = "Files/nyc_taxi/yellow/2019/yellow_tripdata_2019-01.parquet"
green_path  = "Files/nyc_taxi/green/2019/green_tripdata_2019-01.parquet"
y = spark.read.parquet(yellow_path).withColumn("service_type", F.lit("yellow"))
g = spark.read.parquet(green_path).withColumn("service_type", F.lit("green"))
all_cols = sorted(set(y.columns) | set(g.columns))
def align(df, cols):
    for c in cols:
        if c not in df.columns:df = df.withColumn(c, F.lit(None))
    return df.select(*cols)
taxi = align(y, all_cols).unionByName(align(g, all_cols))
taxi.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("dbo.silver_nyc_taxi_trips_v2")
print("silver trips:", spark.table("dbo.silver_nyc_taxi_trips_v2").count())

StatementMeta(, 73fb18c5-dca6-4252-b1a4-225a29c95f92, 4, Finished, Available, Finished)

silver trips: 8368722


In [3]:
from pyspark.sql import functions as F
df = spark.table("dbo.silver_nyc_taxi_trips_v2")
df.groupBy("service_type").count().show()
print(df.columns)

for c in ["tpep_pickup_datetime","tpep_dropoff_datetime","lpep_pickup_datetime","lpep_dropoff_datetime"]:
    if c in df.columns:
        df.select(
            F.count("*").alias("rows"),
            F.sum(F.col(c).isNull().cast("int")).alias(f"{c}_nulls"),
            F.min(c).alias(f"{c}_min"),
            F.max(c).alias(f"{c}_max")
        ).show(truncate=False)

StatementMeta(, 73fb18c5-dca6-4252-b1a4-225a29c95f92, 5, Finished, Available, Finished)

+------------+-------+
|service_type|  count|
+------------+-------+
|      yellow|7696617|
|       green| 672105|
+------------+-------+

['DOLocationID', 'PULocationID', 'RatecodeID', 'VendorID', 'airport_fee', 'congestion_surcharge', 'ehail_fee', 'extra', 'fare_amount', 'improvement_surcharge', 'lpep_dropoff_datetime', 'lpep_pickup_datetime', 'mta_tax', 'passenger_count', 'payment_type', 'service_type', 'store_and_fwd_flag', 'tip_amount', 'tolls_amount', 'total_amount', 'tpep_dropoff_datetime', 'tpep_pickup_datetime', 'trip_distance', 'trip_type']
+-------+--------------------------+------------------------+------------------------+
|rows   |tpep_pickup_datetime_nulls|tpep_pickup_datetime_min|tpep_pickup_datetime_max|
+-------+--------------------------+------------------------+------------------------+
|8368722|672105                    |2001-02-02 14:55:07     |2088-01-24 00:25:39     |
+-------+--------------------------+------------------------+------------------------+

+------

In [4]:
from pyspark.sql import functions as F

df = spark.table("dbo.silver_nyc_taxi_trips_v2")
pickup_ts = F.when(F.col("service_type")=="yellow", F.col("tpep_pickup_datetime")).otherwise(F.col("lpep_pickup_datetime"))
dropoff_ts = F.when(F.col("service_type")=="yellow", F.col("tpep_dropoff_datetime")).otherwise(F.col("lpep_dropoff_datetime"))
df2 = df.withColumn("pickup_ts_utc", pickup_ts).withColumn("dropoff_ts_utc", dropoff_ts).withColumn("trip_day", F.to_date("pickup_ts_utc"))
df_jan = df2.filter((F.col("pickup_ts_utc") >= F.lit("2019-01-01")) & (F.col("pickup_ts_utc") <  F.lit("2019-02-01")))
df_jan.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("dbo.silver_nyc_taxi_trips_2019_01_v2")
print("silver jan trips:", spark.table("dbo.silver_nyc_taxi_trips_2019_01_v2").count())
spark.table("dbo.silver_nyc_taxi_trips_2019_01_v2").groupBy("service_type").count().show()

StatementMeta(, 73fb18c5-dca6-4252-b1a4-225a29c95f92, 6, Finished, Available, Finished)

silver jan trips: 8368072
+------------+-------+
|service_type|  count|
+------------+-------+
|      yellow|7696080|
|       green| 671992|
+------------+-------+



In [5]:
from pyspark.sql import functions as F

t = spark.table("dbo.silver_nyc_taxi_trips_2019_01_v2")
daily = (
    t.groupBy("trip_day")
     .agg(
        F.count("*").alias("trips_cnt"),
        F.sum(F.col("total_amount").cast("double")).alias("revenue_total"),
        F.avg(F.col("total_amount").cast("double")).alias("avg_total_amount"),
        F.avg(F.col("fare_amount").cast("double")).alias("avg_fare_amount"),
        F.avg(F.col("trip_distance").cast("double")).alias("avg_trip_distance")
     )
)
zone_daily = (
    t.groupBy("trip_day", "PULocationID")
     .agg(
        F.count("*").alias("trips_cnt"),
        F.sum(F.col("total_amount").cast("double")).alias("revenue_total"),
        F.avg(F.col("total_amount").cast("double")).alias("avg_total_amount")
     )
)
daily.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable("dbo.silver_nyc_taxi_daily_v2")
zone_daily.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable("dbo.silver_nyc_taxi_zone_daily_v2")
print("daily rows:", spark.table("dbo.silver_nyc_taxi_daily_v2").count())
print("zone_daily rows:", spark.table("dbo.silver_nyc_taxi_zone_daily_v2").count())
spark.table("dbo.silver_nyc_taxi_daily_v2").orderBy("trip_day").show(5, truncate=False)
spark.table("dbo.silver_nyc_taxi_zone_daily_v2").orderBy("trip_day","PULocationID").show(5, truncate=False)

StatementMeta(, 73fb18c5-dca6-4252-b1a4-225a29c95f92, 7, Finished, Available, Finished)

daily rows: 31
zone_daily rows: 7671
+----------+---------+------------------+------------------+------------------+------------------+
|trip_day  |trips_cnt|revenue_total     |avg_total_amount  |avg_fare_amount   |avg_trip_distance |
+----------+---------+------------------+------------------+------------------+------------------+
|2019-01-01|204865   |3480548.659996443 |16.98947433674099 |13.81988275205638 |3.5650036365410873|
|2019-01-02|218637   |3716574.4699959503|16.998835832891736|13.72879937979419 |3.3072943280414586|
|2019-01-03|245896   |3979385.4199947715|16.18320517615078 |13.013668705469275|3.0773931662166327|
|2019-01-04|259212   |4116210.9099940252|15.87970815392044 |12.778005725043885|2.9311595527985412|
|2019-01-05|257316   |3718223.8999953233|14.450029924277244|11.692708964852695|2.7426249047863385|
+----------+---------+------------------+------------------+------------------+------------------+
only showing top 5 rows

+----------+------------+---------+------------